In [1]:
import numpy as np
import collections as cols

class Graph():
    # constructor
    def __init__(self):
        f = open('inputPS01.txt', 'r') # opening file
        self.weight_map = {}   # initialising dict object to hold the file contents

        for i in f:     # manipulating the contents of the file to extract the input
            if(i[0]=='['):
                splitLine=str(i.split())
                temp_str='['+splitLine.rstrip("']").lstrip("['")

                temp_str=temp_str.split(':')
       
                self.weight_map[temp_str[0]]=int(temp_str[1])

        # finding vertices of the NxN matrix
        key_list=[]   
        for element in self.weight_map:
            key_list.append(element)

        max_key=[]
        for key in key_list:
            max_key.append(int(key[1:-1].split(',')[0]))

        max_key.sort()
        max_key[-1]
        self.V=max_key[-1]  # storing the vertice within the class
        self.graph = np.zeros((max_key[-1],max_key[-1]), dtype=int, order='C')  # initialsing matrix to hold values

        for element in self.weight_map:   # adding elements to the graph
            coOrdinates=element[1:-1].split(',')
            self.graph[int(coOrdinates[0])-1][int(coOrdinates[1])-1]=int(self.weight_map[element])

        # finding source and destination coordinates
        f = open('inputPS01.txt', 'r') # opening file
        self.src = np.empty((2))
        self.dest = np.empty((2))
        for i in f:
            if i.lower().find("source node") != -1:
                x = i[13:].rstrip("\n").rstrip(" ").rstrip("]").split(",")
                self.src = np.asarray((int(x[0])-1,int(x[1])-1))
                
        f = open('inputPS01.txt', 'r') # opening file
        for i in f:
            if i.lower().find("destination node") != -1:
                x = i[18:].rstrip("\n").rstrip(" ").rstrip("]").split(",")
                self.dest = np.asarray((int(x[0])-1,int(x[1])-1))
        
        f.close()

    # function to check whether direction is valid or not
    def isValid(self,x, y, visited):
        if ((x >= 0 and y >= 0) and (x < self.V and y < self.V) and ((x, y) not in visited)):
            return True
        else:
            return False
    
    # function to find diagonals of a given index
    def isDiagonal(self,index):
        dia=[(-1,-1),(1,-1),(-1,1),(1,1)]
        index_dia=set()
        for i,j in dia:
            new_x, new_y = (index[0]+i), (index[1]+j)
            if((index[0] >= 0 and index[1] >= 0) and (index[0] < self.V and index[1] < self.V)):
                index_dia.add((new_x, new_y))
                
        return index_dia
    
    # function to find neighbours of a given index
    def isNeighbor(self,index):
        dirs = [(1,0),(-1,0),(0,1),(0,-1)]
        index_dir=set()
        for i,j in dirs:
            new_x, new_y = (index[0]+i), (index[1]+j)
            if((index[0] >= 0 and index[1] >= 0) and (index[0] < self.V and index[1] < self.V)):
                index_dir.add((new_x, new_y))
                
        return index_dir
        
    # implementing breadth first search to find the shortest path
    def bfs(self,graph,src,dest):
        visited = set()  # visited set to hold indexes for visited cells
        visited.add(tuple(src))
        shortest_path = cols.deque() # queue set to hold indexes shortest path route
        shortest_path.append(tuple(src))
        
        q = cols.deque() # queue to hold indexes to explore using bfs
        dirs = [(1,0),(-1,0),(0,1),(0,-1)]
        
        x = src[0]
        y = src[1]
        
        q.append((graph[x][y],(x,y)))
        
        while q: # while loop will run as long as there are cells to explore and q is not empty
            cost, temp = q.popleft()
            x,y = temp[0],temp[1]
            if(x == dest[0] and y == dest[1]):
                return shortest_path
            
            min_cost=10e7
            index=tuple()
            
            neighbour_set=set()
            
            for i,j in dirs:
                new_x, new_y = (x+i), (y+j)  # unexplored directions
                
                if self.isValid(new_x, new_y, visited): # checking whether whether obtaines directions as vaild
                    q.append((graph[new_x][new_y],(new_x, new_y))) # if valid append to q
                    visited.add((new_x, new_y))                   # if valid add to visited set
                    neighbour_set.add((new_x, new_y))            # if valid add to neighbours set
                    
                    if(graph[new_x][new_y]<min_cost):         # checking which direction has min cost
                        min_cost=graph[new_x][new_y]
                        index=(new_x, new_y)

            if(index != () and index not in shortest_path):
                neighbour_set.remove(index)
                if(index not in self.isDiagonal(shortest_path[-1]) and index in self.isNeighbor(shortest_path[-1])):
                    shortest_path.append(index)  # adding to shortest path only if cell have min cost and
                                                    # it is a neighbour of previous index
                
            
            for i in neighbour_set:
                if(i != () and i in shortest_path):
                    shortest_path.remove(i)   # removing any outliers from shortest path
                    
    def shortestPath(self):

        q1 = self.bfs(self.graph,self.src,self.dest)  # getting shortest path from bfs function
        q2 = cols.deque()
        min_time = 0
        min_time_str = str()
        
        for i in q1:
            q2.append((i[0]+1,i[1]+1))
        
        # exporting output to outputPS01.txt file
        with open('outputPS01.txt', 'w') as f:
            f.write('Shortest route from the House to reach the Hospital is:\n')
            for i in q2:
                f.write(str(list(i))+'\n')
                min_time = min_time + self.weight_map[str(list(i)).replace(" ", "")]
                min_time_str = min_time_str + ' + ' + str(self.weight_map[str(list(i)).replace(" ", "")])
            
            f.write('Minimum time : '+str(min_time)+'\n')
            f.write(min_time_str[7:-4])
            f.close()

g=Graph()
g.shortestPath()